#### Multilayer percepron: A class of fully connected layers also known as FeedForward Neural Networks

#####  Feedforward Network:  
is a function approximation machine that are designed to achieve statistical generalization, occasionally drawn some insights from what we known about the brain, rather than as models of brain function.

#### Packages selection
- The first things is to import all the neccesary packages needed for this project

In [14]:
import time
import numpy as np 
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

# select GPU when cuda is available
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

#### Settings
- Configure the device
- define all the hyperparameters to be used and needs to be tuned to achive a better accuracy
- Load and explore the data

In [15]:
# device
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")


# Hyperparameters
random_seed = 1 # for generating random number
learning_rate = 0.1
num_epochs = 10
batch_size = 64 #power of 2 is prefferable

# Model Architecture parameters
num_features = 784 
num_hidden_1 = 128 
num_hidden_2 = 256
num_classes = 10


# dataset -> MNIST
# Note: tranforms.ToTensor() scale images to 0-1 range

train_dataset = datasets.MNIST(root='data',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)

test_dataset = datasets.MNIST(root='data',
                             train=False,
                             transform=transforms.ToTensor())

train_loader = DataLoader(dataset=train_dataset,
                         batch_size=batch_size,
                         shuffle=False)

# check the dataset
for images, labels in train_loader:
    print('Image batch dimension', images.shape)
    print('image lable dimension', labels.shape)
    break

Image batch dimension torch.Size([64, 1, 28, 28])
image lable dimension torch.Size([64])


#### Define the architecture of the model such as
- The number of input layers; which is determined by the features of the data
- Number of total hidden layers in the model (iterative)
- Number of hidden units in each layers (iterative)
- The output layer node units is determined by the intended outcome to achieve

### Here: we build a 3 layers multilayer perceptron i.e 2 hidden layers and 1 output layer
#####  Note: We don't count the input layer as part of the layers.

In [17]:
"""
Architecture
X -> Linear -> Relu -> Linear -> Relu -> Linear -> softmax -> y
"""
class MultiLayerPerceptron(nn.Module):
    def __init__(self, num_features, num_classes):
        
        """
        Basic defintion of each layers parameters
        """
        super(MultiLayerPerceptron, self).__init__()
        
        # 1st Hidden Layer
        self.linear_1 = nn.Linear(num_features, num_hidden_1)
        
        # 2nd Hidden layer
        self.linear_2 = nn.Linear(num_hidden_1, num_hidden_2)
        
        # output layer
        self.linear_out = nn.Linear(num_hidden_2, num_classes)
        
    def forward(self, x):
        """
        Link all the layers together
        """
        out = self.linear_1(x)
        out = F.relu(out)
        out = self.linear_2(out)
        out = F.relu(out)
        logits = self.linear_out(out)
        probas = F.log_softmax(logits, dim=1)
        return logits, probas

#### Loss function and optimizer
- Instantiate the model
- define the specific Loss function to be used either cross entropy, MSELoss, etc
- define the optimization algorithm to be used either SGD, Adam, RMSprop, Momentum et

In [18]:
torch.manual_seed(random_seed) # generate a random number
model = MultiLayerPerceptron(num_features=num_features,
                            num_classes=num_classes)

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#### compute accuracy
- A function to compute train and test accuracy

In [13]:
def compute_accuracy(net, data_loader):
    net.eval()
    correct_pred, num_exmples = 0, 0
    with torch.no_grad():
        for features, labels in data_loader:
            features = features.view(-1, 28*28).to(device)
            labels = labels.to(device)
            logits, probas = net(features)
            _, predicted_labels = torch.max(probas, 1)
            num_examples += labels.size(0)
            correct_pred += (predicted_labels == labels).sum()
        return correct_pred.float()/num_examples * 100

#### Training a model requires the following steps
- Reset all the gradients to zero (0)
- Make a forward pass (make a prediction)
- Calculate the loss
- Perform back propagation
- Update all the parameters (weight and biases)

In [ ]:
# Train model
total_step = len(train_loader)
start_time = time.time()

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # Logging
        if not i % 50:
            print('Epoch: %03d/%03d | Batch %03d/%03d | Loss: %.4f'
                 %(epoch+1, num_epochs, i, total_step, loss))
    with torch.set_grad_enabled(False):
        print('Epoch: %03d/%03d training accuracy: %.2f%%' %(
        epoch+1, num_epochs, compute_accuracy(model, train_loader)))
        
    print('Time elapsed: %.2f min' % ((time.time() - start_time) / 60))
print('Total Training Time: %.2f min ' % ((time.time() - start_time) / 60))
        